In [6]:
import os
os.chdir('/home/dan/gulon') 

In [7]:
import argparse
#import tools.find_mxnet
import mxnet as mx
import os
import sys
import logging
import opt
from model.hg_attention import createModel
from iterator import  hgIter

In [8]:
def train_net():
    ctx = [mx.gpu(int(i)) for i in opt.gpus.split(',') if i.strip()]
    ctx = [mx.cpu()] if not ctx else ctx
    logging.basicConfig()
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    train_iter = hgIter(path_imgrec=opt.train_path, data_shape=opt.data_shape, batch_size=opt.batch_size,max_rotate_angle=30,
                       mean_pixels=opt.mean_pixels,color_jitter=20)


    # if opt.val_path:
    #     val_iter = hgIter(path_imgrec=opt.val_path, data_shape=opt.data_shape, batch_size=opt.batch_size,max_rotate_angle=30,
    #                     mean_pixels=opt.mean_pixels,color_jitter=20)
    # else:
    #     val_iter = None
    data =  mx.sym.Variable('data')
    net = createModel(data)
    #net = mx.symbol.Convolution(data=data, num_filter=64, kernel=7, stride=1, pad=3, name="conv1")
    #net = mx.symbol.FullyConnected(data=net,num_hidden=46,name='fc')
    #net = mx.sym.SoftmaxOutput(data=net, name='softmax')
    mod = mx.mod.Module(symbol=net,
                        context=ctx,
                        logger=logger,
                        label_names=('label',)
                        )
    print(train_iter.provide_data)
    print(train_iter.provide_label)
    mod.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)
    mod.init_params(initializer= mx.init.Xavier(rnd_type='gaussian', factor_type="in", magnitude=2) )
    mod.init_optimizer(optimizer=mx.optimizer.RMSProp(learning_rate=2.5e-4))
    metric = mx.metric.create(mx.metric.MSE())

    for epoch in range(5):
        train_iter.reset()
        metric.reset()
        for batch in train_iter:
            mod.forward(batch, is_train=True)  # compute predictions
            mod.update_metric(metric, batch.label)  # accumulate prediction accuracy
            mod.backward()  # compute gradients
            mod.update()  # update parameters
        print('Epoch %d, Training %s' % (epoch, metric.get()))

In [9]:
train_net()

[DataDesc[data,(2, 3, 256, 256),<class 'numpy.float32'>,NCHW]]
[('label', (2, 14, 64, 64))]


/home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/mxnet/module/base_module.py:53: UserWarning: You created Module with Module(..., label_names=['label']) but input with name 'label' is not found in symbol.list_arguments(). Did you mean one of:
	data
  warnings.warn(msg)


RuntimeError: simple_bind error. Arguments:
data: (2, 3, 256, 256)
label: (2, 14, 64, 64)
Error in operator conv1: [16:28:21] src/operator/./convolution-inl.h:446: Check failed: dshp.ndim() == 3U (4 vs. 3) Input data should be 3D in batch-num_filter-x

Stack trace returned 10 entries:
[bt] (0) /home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x1f7a8c) [0x7f22b2aa8a8c]
[bt] (1) /home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x11c0ab0) [0x7f22b3a71ab0]
[bt] (2) /home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x113aff7) [0x7f22b39ebff7]
[bt] (3) /home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/mxnet/libmxnet.so(+0xf7fae7) [0x7f22b3830ae7]
[bt] (4) /home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/mxnet/libmxnet.so(+0xf817cb) [0x7f22b38327cb]
[bt] (5) /home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/mxnet/libmxnet.so(+0xf69b42) [0x7f22b381ab42]
[bt] (6) /home/d